In [1]:
import os
import sys

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(PROJECT_ROOT)

from src.paths import RAW_AUDIO_DIR, PROCESSED_AUDIO_DIR
print("RAW_AUDIO_DIR:", RAW_AUDIO_DIR)
print("PROCESSED_AUDIO_DIR:", PROCESSED_AUDIO_DIR)

RAW_AUDIO_DIR: c:\Users\user\OneDrive\Desktop\portfolio\portfolio_projects\DRI_SWAASA\data\raw\audio
PROCESSED_AUDIO_DIR: c:\Users\user\OneDrive\Desktop\portfolio\portfolio_projects\DRI_SWAASA\data\processed\audio


In [2]:
import librosa
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis

In [3]:
def extract_audio_features(audio_path, filename, sr=22050, n_mfcc=13):
    y, sr = librosa.load(audio_path, sr=sr)
    features = {}

    features["filename"] = filename
    features["recording_length"] = librosa.get_duration(y=y, sr=sr)

    # RMS
    rms = librosa.feature.rms(y=y)[0]
    features["rms_power"] = np.mean(rms)

    # ZCR
    zcr = librosa.feature.zero_crossing_rate(y=y)[0]
    features["zero_crossing_rate"] = np.mean(zcr)

    # Crest Factor
    peak = np.max(np.abs(y))
    features["crest_factor"] = peak / (np.sqrt(np.mean(y**2)) + 1e-8)

    # MFCCs
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    for i in range(n_mfcc):
        features[f"mfcc_{i+1}_mean"] = np.mean(mfcc[i])
        features[f"mfcc_{i+1}_std"] = np.std(mfcc[i])

    # Spectrum
    spectrum = np.abs(librosa.stft(y))
    freqs = librosa.fft_frequencies(sr=sr)
    psd = np.mean(spectrum**2, axis=1)

    features["spectral_centroid"] = np.mean(
        librosa.feature.spectral_centroid(y=y, sr=sr)
    )
    features["spectral_rolloff"] = np.mean(
        librosa.feature.spectral_rolloff(y=y, sr=sr)
    )
    features["spectral_spread"] = np.mean(
        librosa.feature.spectral_bandwidth(y=y, sr=sr)
    )
    features["spectral_flatness"] = np.mean(
        librosa.feature.spectral_flatness(y=y)
    )

    features["spectral_skewness"] = skew(psd)
    features["spectral_kurtosis"] = kurtosis(psd)
    features["spectral_std"] = np.std(psd)

    slope = np.polyfit(freqs, psd, 1)[0]
    features["spectral_slope"] = slope

    features["spectral_decrease"] = np.mean(
        (psd[1:] - psd[0]) / (freqs[1:] + 1e-8)
    )

    return features


In [ ]:
features_list = []

audio_files = [
    f for f in os.listdir(RAW_AUDIO_DIR)
    if f.lower().endswith((".wav", ".mp3"))
]

print("Audio files found:", audio_files)

for file in audio_files:
    path = os.path.join(RAW_AUDIO_DIR, file)
    feats = extract_audio_features(path, file)
    features_list.append(feats)

df = pd.DataFrame(features_list)
df.head()


Audio files found: ['C01.mp3', 'C02.mp3', 'C03.mp3', 'C04.mp3', 'C05.mp3', 'C06.mp3', 'C07.mp3', 'C08.mp3', 'C09.mp3', 'C10.mp3']


In [ ]:
output_csv = os.path.join(PROCESSED_AUDIO_DIR, "extracted_features.csv")
df.to_csv(output_csv, index=False)

print("Saved to:", output_csv)


Saved to: c:\Users\user\OneDrive\Desktop\portfolio\portfolio_projects\DRI_SWAASA-main\data\processed\audio\extracted_features.csv
